In [18]:
%matplotlib inline

In [19]:
import fipy as fp

In [20]:
from matplotlibElPhFViewer import MatplotlibElPhFViewer
def viewElPhF(filename, L="3.2 nm", nx=1200):
    L = fp.Variable(L)
    dx = L / nx
    mesh = fp.Grid1D(nx=nx, dx=dx/L)

    xi = fp.CellVariable(mesh=mesh, name=r"$\xi$")
    phi = fp.CellVariable(mesh=mesh, name=r"$\phi$")  
    interstitials = [
        fp.CellVariable(mesh=mesh, name="$e^-$")
    ]

    substitutionals = [
        fp.CellVariable(mesh=mesh, name="$M^+$"),
        fp.CellVariable(mesh=mesh, name="$A^-$")
    ]

    N = fp.CellVariable(mesh=mesh, name="$N$")
    
    components = interstitials + substitutionals + [N]
    
    for j in components:
        j.Y = fp.CellVariable(mesh=mesh, name=j.name)

    interstitials[0].z   = -1
    substitutionals[0].z = +2
    substitutionals[1].z = -2
    N.z                  = 0

                         
    values = fp.numerix.loadtxt(filename, skiprows=1, unpack=True)
    
    xi.setValue(values[1])
    phi.setValue(values[2])
    
    _start = 3
    for j, mu in zip(interstitials + substitutionals,
                     values[_start:_start+len(interstitials + substitutionals)]):
        j.setValue(mu)

    _start = 3 + len(interstitials + substitutionals)
    for j, Y in zip(components,
                     values[_start:_start+len(components)]):
        j.Y.setValue(Y)

    charge = fp.CellVariable(mesh=mesh, value=N.z)
    for i in interstitials + substitutionals:
        charge += i.z * i.Y

    viewer = MatplotlibElPhFViewer(phase=xi,
                           potential=phi,
                           components=[i.Y for i in components],
                           charge=charge,
                           potentials=[i - i[...,0] for i in interstitials + substitutionals],
                           limits={
                               'phasemax':1.,
                               'phasemin':0.,
                           })


## elphf0226.py

In [ ]:
!python elphf0226.py --galvani_potential="0.0 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="0.0_V.tsv" 

In [ ]:
viewElPhF("0.0_V.tsv")

In [ ]:
!python elphf0226.py --galvani_potential="0.5 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="0.5_V.tsv" 

In [ ]:
viewElPhF("0.5_V.tsv")

In [ ]:
!python elphf0226.py --galvani_potential="-0.1 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="-0.1_V.tsv" 

In [ ]:
viewElPhF("-0.1_V.tsv")

Negative Galvani potentials do not converge well. Prone to negative electron concentration, as a result of negative $Y_m$

In [ ]:
!python elphf0226.py --galvani_potential="-0.2 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="-0.2_V.tsv" 

In [ ]:
viewElPhF("-0.2_V.tsv")

Restart from previous solution at different Galvani potential does not result in "correct" equilibrium. Stabilizes at something with non-uniform electrochemical potentials. Possibly because equations are not well-coupled?

In [ ]:
!python elphf0226.py --galvani_potential="-0.2 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="-0.2_Vb.tsv" --restart="0.0_V.tsv" 

In [ ]:
viewElPhF("-0.2_Vb.tsv")

In [ ]:
"{0} {1} {2}".format(1.23, 3.14, 5)

In [ ]:
!python elphf0226.py params.json

In [21]:
%%bash

smt configure --executable=python
smt configure --labelgenerator=uuid
smt configure --on-changed=store-diff
smt configure --addlabel=parameters


Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly


In [ ]:
!mkdir Data

In [ ]:
!ls

In [ ]:
!smt run --main=elphf0226.py params.json

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,2):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l"

In [ ]:
import json
import pandas

In [ ]:
!smt export
with open(".smt/records_export.json") as ff:
    data = json.load(ff)
    
df = pandas.DataFrame(data)

In [ ]:
df[["label","outcome","parameters"]]

In [ ]:
df.keys()

In [ ]:
!ls Data

In [ ]:
!ls Data/ce503af2e3b1/

In [ ]:
!cat Data/ce503af2e3b1/test.txt

In [ ]:
for r in df[["parameters"]].iterrows():
    print r[1]["parameters"]

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.01 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.10 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="10.0 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=800 concentration="0.25 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=600 concentration="0.25 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=300 concentration="0.25 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=120 concentration="0.25 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=12000 concentration="0.25 mol/l"

In [ ]:
import timeit

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0237.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l" --tag"0237" 

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 1.99957273847e-09 1.99999999308
$M^+$ 0.0045 0.99999999673
$A^-$ 0.00449999900021 1.87483286232e-10
$N$ 0.991000001 3.08279723394e-09
$m$ 1.000000002 2.99999999308
deltaV -0.693132797286
$e^-$ -20.7234651093 -20.7234651093
$M^+$ -4.00837154399 -4.00837154399
$A^-$ -6.78090295531 -6.78090295531
dt 0.0732421875
0.0 0.655212210675 -0.00471784553694
0.0 0.543689759255 -0.00660166924017
0.0 0.552850170925 -0.00719504322247
0.0 0.540128215863 -0.0070729211874
0.0 0.543682956284 -0.00712762262758
0.0 0.542219556953 -0.00710820426324
0.0 0.542758395425 -0.00711567157808
0.0 0.542552244663 -0.00711285014152
0.0 0.542600613786 -0.00711351483371
Record label for this run: '0b6231452b1b'
Data keys are [test.txt(4b78d72eea82d91984b96bd344cfaa5319ecad6e [2016-07-13 11:24:16]), output.tsv(b8b7b09909c51212de880f4056b85bc36

In [4]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0237.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l" outer_sweeps=5 --tag"0237"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 9.99786366371e-10 1.999999982
$M^+$ 0.018 0.999999994001
$A^-$ 0.0179999995001 2.99973315489e-09
$N$ 0.9640000005 2.99880577604e-09
$m$ 1.000000001 2.999999982
deltaV 1.43832910208e-05
$e^-$ -20.7234651111 -20.7234651111
$M^+$ -3.98074830381 -3.98074830381
$A^-$ -3.98069079842 -3.98069079842
dt 0.0732421875
0.0 0.667439558399 -0.00120542322487
0.0 0.54261116702 -0.00176144337403
0.0 0.539209983966 -0.00195039768518
0.0 0.537470462842 -0.00193898752235
Record label for this run: '88b12685e633'
Data keys are [test.txt(f2a59bb4994ab62fa098ffa732806882aba43d05 [2016-07-15 09:21:04]), output.tsv(96c6ec354a48ed5c2304f9583a653b710732f26f [2016-07-15 09:21:13])]
Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
b

In [ ]:
#separates files from elphf0238 simulations
df0238 = df4[(df4["main_file"]=="elphf0238.py")]
df0238for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0238.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l" --tag"0238"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 2.00010268315e-18 2.0
$M^+$ 0.0045 1.0
$A^-$ 0.0045 1.87539108722e-19
$N$ 0.991 3.08360995694e-18
$m$ 1.0 3.0
deltaV -0.693132796769
$e^-$ -41.4464651081 -41.4464659498
$M^+$ -4.0083693007 -4.00837154402
$A^-$ -6.78086885007 -6.78090273109
dt 0.0732421875
0.0 0.639128252595 -0.00585554604291
0.0 0.595467908085 -0.00291403722695
0.0 0.493354122738 -0.00423463748843
0.0 0.503717669935 -0.00261521103154
0.0 0.458626299895 -0.00540899871672
0.0 0.516001891611 -0.00496408850149
0.0 0.516701081198 -0.00427559930931
0.0 0.519322617455 -0.00332028562838
0.0 0.477232351016 -0.00477169516573
Record label for this run: 'eba1be1d110c'
Data keys are [test.txt(91c1d0dd187348052cd4dc0b40c3a138a947e8ee [2016-07-18 08:41:11]), output.tsv(cc7dd993411c21cd5cb8d7cf9d86bfd1a1eb51f3 [2016-07-18 08:41:19])]
Multiple versions foun

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0238.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l" --tag"0238"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0239.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l" --tag"0239"

In [6]:
for galvani in fp.numerix.linspace(0.207070707071,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0239.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l" --tag"0239"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 1.00015500693e-09 0.999999994001
$M^+$ 0.018 0.999999996001
$A^-$ 0.0179999989998 1.99969185528e-09
$N$ 0.964000001 1.99920385773e-09
$m$ 1.000000001 1.999999994
deltaV -3.63405265799e-05
$e^-$ -12.6595464923 -12.6595464923
$M^+$ -12.0442838885 -12.0442838885
$A^-$ 4.0828447574 4.0828447574
dt 0.0732421875
8.06360069124 0.551909694703 0.0300058863146
8.06360069124 0.425908031343 0.0290077770954
8.06360069124 0.419123205685 0.0289360540215
8.06360069124 0.418830663166 0.0289321384667
8.06360069124 0.418813768363 0.0289319040632
8.06360069124 0.418812747909 0.0289318898708
8.06360069124 0.418812686156 0.0289318890107
8.06360069124 0.41881268242 0.0289318889573
8.06360069124 0.418812682187 0.0289318889531
Record label for this run: '52220dd40bd2'
Data keys are [test.txt(a71f7ff68a490baecb75d955fad4518ca5ef9cbb

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0240.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l" --tag"0240"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0240.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l" --tag"0240"

In [10]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0237.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.01 mol/l" outer_sweeps=10 --tag"0237"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 9.99786381309e-09 1.99999999378
$M^+$ 0.00018 0.99999999689
$A^-$ 0.000179995001068 2.99964989098e-13
$N$ 0.999640004999 3.10967448568e-09
$m$ 1.00000001 2.99999999378
deltaV -2.30257071368
$e^-$ -20.7234651091 -20.7234651091
$M^+$ -4.01705221989 -4.01705221989
$A^-$ -13.2273628469 -13.2273628469
dt 0.0732421875
0.0 0.649848741548 -0.00713150222285
0.0 0.537453198947 -0.00857751322834
0.0 0.534218732288 -0.00875711295755
0.0 0.52780071868 -0.00880970747014
0.0 0.523584356168 -0.00883499889274
0.0 0.522368149768 -0.0089332014738
0.0 0.51935189948 -0.00893703187873
0.0 0.524140925189 -0.00888986768942
0.0 0.52290467182 -0.00911463259273
Record label for this run: '07f19f1e7a69'
Data keys are [test.txt(da394c072f6b322695e0807192ec34594889516d [2016-07-20 08:24:37]), output.tsv(1ab3948d78555a4b53540c352b39a8c60

In [11]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0237.py params.json galvani_potential="$galvani V" ncells=1200 concentration="10.0 mol/l" outer_sweeps=7 --tag"0237"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 3.16160162021e-10 1.99999879613
$M^+$ 0.18 0.999999698036
$A^-$ 0.179999999842 2.99973175593e-07
$N$ 0.640000000158 1.99090761286e-09
$m$ 1.00000000032 2.99999879613
deltaV 1.15130688047
$e^-$ -20.7234653088 -20.7234653088
$M^+$ -3.57112508664 -3.57112508664
$A^-$ 1.03410243434 1.03410243434
dt 0.0732421875
0.0 0.661984870726 0.0127155550238
0.0 0.514483933823 0.011755079434
0.0 0.507786663953 0.0115888921876
0.0 0.506995359195 0.0115787652776
0.0 0.506955503435 0.0115770453764
0.0 0.506945783582 0.0115769764028
Record label for this run: '14b05d4e0164'
Data keys are [test.txt(38123186f4a3c14c880c8fdd68537679feb1f6a7 [2016-07-20 12:14:30]), output.tsv(53e8c6c1c53a201d00a8a75bb01f87ac5d8da60d [2016-07-20 12:14:38])]
Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use 

In [12]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0237.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.10 mol/l" outer_sweeps=7 --tag"0237"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 3.1616021061e-09 1.99999999368
$M^+$ 0.0018 0.99999999687
$A^-$ 0.0017999984192 2.99973060554e-11
$N$ 0.996400001581 3.09959552242e-09
$m$ 1.00000000316 2.99999999368
deltaV -1.15127816378
$e^-$ -20.7234651092 -20.7234651092
$M^+$ -4.01380579252 -4.01380579252
$A^-$ -8.61891932585 -8.61891932585
dt 0.0732421875
0.0 0.651098564081 -0.00603352922373
0.0 0.536663437471 -0.00745737603082
0.0 0.532402153526 -0.00772197817857
0.0 0.531942776391 -0.00828990081364
0.0 0.55586434781 -0.00872260198226
0.0 0.544954325213 -0.00865936902535
Record label for this run: '5201b418dcaf'
Data keys are [test.txt(8b061a840dad7b175e35d7101aaed5a5b2b42a43 [2016-07-20 15:46:14]), output.tsv(b0070d79495e674a4d30bc70b151c39bd8b10613 [2016-07-20 15:46:22])]
Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If 

In [13]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0239.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.01 mol/l" outer_sweeps=10 --tag"0239"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 1.00015520691e-07 0.999999997926
$M^+$ 0.00018 0.999999997927
$A^-$ 0.000179899984479 1.9985804622e-13
$N$ 0.999640100016 2.07311633637e-09
$m$ 1.00000010002 1.99999999793
deltaV -4.60520662549
$e^-$ -20.7231471816 -20.7231471816
$M^+$ -4.01698711682 -4.01698711682
$A^-$ -13.227956164 -13.227956164
dt 0.0732421875
0.0 0.66948594294 -0.0063234067273
0.0 0.527418996443 -0.0073608906587
0.0 0.520968713083 -0.00747901558145
0.0 0.520032032231 -0.00749131100045
0.0 0.519975688151 -0.00749271601819
0.0 0.519962265628 -0.00749286024625
0.0 0.519961931855 -0.00749287772947
0.0 0.519961715203 -0.007492879537
0.0 0.519961714584 -0.0074928800655
Record label for this run: '91451be87ede'
Data keys are [test.txt(410d63e9fffec557feab61ffaf721b298568a3ab [2016-07-20 18:57:20]), output.tsv(ac49085c04103d638cb0a06fcf771e82f

In [14]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0239.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.1 mol/l" outer_sweeps=10 --tag"0239"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 1.00015502684e-08 0.999999997894
$M^+$ 0.0018 0.999999997914
$A^-$ 0.00179998999845 1.99968082324e-11
$N$ 0.996400010002 2.06639702045e-09
$m$ 1.00000001 1.99999999789
deltaV -2.30262144249
$e^-$ -20.7231471816 -20.7231471816
$M^+$ -4.01374068604 -4.01374068604
$A^-$ -8.61898912744 -8.61898912744
dt 0.0732421875
0.0 0.670537956423 -0.00602383656586
0.0 0.528277943558 -0.00701514728276
0.0 0.52162405607 -0.00713131075144
0.0 0.520707425415 -0.00714326137718
0.0 0.520645257099 -0.00714465153641
0.0 0.520632590228 -0.00714479132517
0.0 0.520632090652 -0.00714480861975
0.0 0.520631899118 -0.00714481022299
0.0 0.520631895756 -0.00714481046688
Record label for this run: '0dedb76e217c'
Data keys are [test.txt(1c14c498313d09a5121eed967ef5f00c5006eaed [2016-07-20 22:43:40]), output.tsv(7d1437c7f77bd2e4c779ad9857f7e6

In [15]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0239.py params.json galvani_potential="$galvani V" ncells=1200 concentration="10.0 mol/l" outer_sweeps=10 --tag"0239"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
/data/ame2/miniconda/envs/fipy/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
L = 3.2 nm
barrier 145.295286758
$e^-$ 3e-06 2.0
$M^+$ 0.18 1.0
$A^-$ 0.18 3e-06
$N$ 1.0 3e-06
$m$ 1.360003 3.000006
deltaV 0.0
$e^-$ -20.4354671081 -13.0243851749
$M^+$ -11.2943017307 -1.71479842809
$A^-$ -3.981 -1.71479842809
dt 0.0732421875
0.0 0.673103817836 -0.0249646661771
0.0 0.538638806641 -0.00513969985657
0.0 0.520725845907 -0.00265159764362
0.0 0.517136531052 -0.000198113003226
0.0 0.514260394341 0.000378326947446
0.0 0.513300594491 0.000197933757656
0.0 0.513186967279 0.00012763889419
0.0 0.513072731735 5.18504204261e-05
0.0 0.512875235166 -9.05462277943e-05
Record label for this run: 'f2

In [ ]:
for galvani in fp.numerix.linspace(0,0.15,30):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0238.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l" outer_sweeps=15 --tag"0238"

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 2.00010268315e-18 2.0
$M^+$ 0.0045 1.0
$A^-$ 0.0045 1.87539108722e-19
$N$ 0.991 3.08360995694e-18
$m$ 1.0 3.0
deltaV -0.693132796769
$e^-$ -41.4464651081 -41.4464659498
$M^+$ -4.0083693007 -4.00837154402
$A^-$ -6.78086885007 -6.78090273109
dt 0.0732421875
0.0 0.639128252595 -0.00585554604291
0.0 0.595467908085 -0.00291403722695
0.0 0.493354122738 -0.00423463748843
0.0 0.503717669935 -0.00261521103154
0.0 0.458626299895 -0.00540899871672
0.0 0.516001891611 -0.00496408850149
0.0 0.516701081198 -0.00427559930931
0.0 0.519322617455 -0.00332028562838
0.0 0.476367443508 -0.00357598690379
0.0 0.477232351016 -0.00477169516573
0.0 0.502204199082 -0.00695784783037
0.0 0.587850420545 -0.00497704059035
0.0 0.519839294367 -0.00482093607241
0.0 0.481758794958 -0.00690076863732
Record label for this run: '11cf51c23cf7'
Da

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0238.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.01 mol/l" outer_sweeps=10 --tag"0238"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0238.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.10 mol/l" outer_sweeps=10 --tag"0238"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0238.py params.json galvani_potential="$galvani V" ncells=1200 concentration="10.0 mol/l" outer_sweeps=10 --tag"0238"